## Real-Time News Data Collection using Python

### Step 1: Fetch Live News Using NewsAPI
Here’s the function we can use to fetch news headlines:

## pip install dotenv wordcloud

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

NEWS_API_KEY=4349a86718a74289a0ca0c5f7025b696


# load API key from .env file
load_dotenv()
API_KEY = os.getenv("NEWS_API_KEY")

# dates for the last 2 days
to_date = datetime.utcnow().date()
from_date = to_date - timedelta(days=2)

QUERY = 'India Pakistan war OR conflict OR border OR attack OR tension'
LANG = 'en'
PAGE_SIZE = 100  # max per page

def fetch_news(query, from_date, to_date, api_key, page_size=100):
    url = 'https://newsapi.org/v2/everything'
    all_articles = []
    for page in range(1, 6):  # try up to 5 pages
        params = {
            'q': query,
            'from': from_date,
            'to': to_date,
            'language': LANG,
            'pageSize': page_size,
            'page': page,
            'sortBy': 'publishedAt',
            'apiKey': api_key
        }
        response = requests.get(url, params=params)
        data = response.json()

        if data.get("status") != "ok":
            print("Error:", data.get("message"))
            break

        articles = data.get("articles", [])
        if not articles:
            break
        all_articles.extend(articles)
    return all_articles

SyntaxError: invalid decimal literal (2158225172.py, line 7)

### Step 2: Store the News Articles in a CSV

In [11]:

# fetch articles

articles = fetch_news(QUERY, from_date.isoformat(), to_date.isoformat(), API_KEY)

# convert to dataframe

df = pd.DataFrame([{

    'title': article['title'],

    'description': article['description'],

    'source': article['source']['name'],

    'published_at': article['publishedAt'],

    'url': article['url'],

    'content': article['content']

} for article in articles])


# save to CSV

df.to_csv("india_pakistan_conflict_news.csv", index=False)

print(f"{len(df)} articles saved to 'india_pakistan_conflict_news.csv'")

Error: Your API key is missing. Append this to the URL with the apiKey param, or use the x-api-key HTTP header.
0 articles saved to 'india_pakistan_conflict_news.csv'
